# WEB SCRAPING
  
Es una técnica para extraer y almacenar datos de una o varias páginas web con el fin de analizarlos o manipularlos en otros medios, para la cual se utilizan bots para extraer los datos y contenidos de las webs. 

### Importación de librerías.

* Requests realiza la petición al servidor.

* BeautifulSoup analizar documentos HTML.

* Pandas podemos representar datos tabulares con columnas con etiquetas y filas y series temporales.

In [ ]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

Se realiza un request de la url_inicial de la revista, url_root nos ayuda a genera un url completa a partir de la url raíz para esto se utiliza la librería urljoin.

La revista que se está scrapendo es la siguiente http://www.revistas.unam.mx/front/ 

In [ ]:
url_inicial='http://revistas.unam.mx/index.php/entreciencias/issue/archive'
url_root= 'http://revistas.unam.mx/index.php/entreciencias/issue/archive'
r = requests.get(url_inicial)

In [ ]:
soup = BeautifulSoup(r.text, 'html.parser')

### Obtención de los primeros volúmenes.

Se realiza la búsqueda para obtener los urls de cada uno de los volúmenes o archivos de la revista.

In [ ]:
box = soup.find('div', id='issues')
volu=box.findAll('h4')
vol = [x.find('a').get('href')for x in volu]
vol

### Obtención de los segundos urls.

Se realiza una lista en la que la variable *vol* que contiene las urls de cada uno de los volúmenes publicados de la revista, con estos urls obtenidos,  con la variable *vol* se implementa un ciclo en el cual se utiliza la variable *vol2* de la cual se van acumulando las siguientes urls.

In [ ]:
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.findAll('div', id='issueCoverImage') 
    #volumen1=box1.findAll('h3',class_='title')
    vol1 = [x.find('a').get('href')for x in box1]
    #vol1=[urljoin(url_root,p) for p in vol1]
    vol2+=vol1

In [ ]:
vol2

### Obtención de las ulrs de los artículos.

Algunas revistas necesitan más búsquedas para llega a los artículos ya que algunos manejan primero los volúmenes o archivos y de eso archivos se dirigen a un urls que es la tabla contenidos, donde se encuentran los artículos, por lo cual se comienza a realizar la misma búsqueda que en *vol2*.

Se implementa otro ciclo con *vol2*, para esto se utiliza la variable de *vol3* que recolecta las urls de los artículos, para comenzar con su scrapeo.

In [ ]:
vol3 =[]
for b in vol2: 
    url_inicial2=b 
    r2 = requests.get(url_inicial2)
    soup2 = BeautifulSoup(r2.text, 'html.parser')
    box2 = soup2.find('div', id='content') 
    volumen2=box2.findAll('div',class_='tocTitle')
    vol2 = [o.find('a').get('href')for o in volumen2]
    #vol1=[urljoin(url_root,p) for p in vol1]
    vol3+=vol2

In [ ]:
vol3

En esta función variable sopa se utiliza para tener las paginaciones de la revista y la variable url es guardan cada uno de url que ya se obtuvieron de *vol3*.

In [ ]:
def get_url_items(sopa, url): 
    box2 = soup2.find('div', id='content') 
    volumen2=box2.findAll('div',class_='tocTitle')
    vol2 = [x.find('a').get('href')for x in volumen2]
    #vol=[urljoin(url_root,i) for i in vol]
    return vol3

Ahora se va acumulando cada uno de los links e ir iterando en cada una de las páginas, para traer cada uno de los links que se van a ir scrapeando.

In [ ]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break
    url_inicial = urljoin(url_inicial, next_a[0].get('href'))

Obtiene el número de los artículos que se encontraron en los primeros volúmenes de la revista.

In [ ]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

Toma uno a uno de los links donde se encuentra la información de cada artículo para Scrapearlos.

In [ ]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

### Scrapeo de las urls de los ariculos.

Mediante esta función revisa cada uno de los links de los artículos en los cuales se encuentra la información
del artículo, por lo cual se aplican cada uno de los métodos, que nos van a obtener lo que estamos requiriendo de dicho artículo.

In [ ]:
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tr='Entreciencias: Diálogos en la Sociedad del Conocimiento'
    a='Multidisciplinaria'
    te='Multidisciplinaria'
    s_item=BeautifulSoup(r.text,'html.parser')
    tituloR=tr
    if tituloR:
        content_book['Titulo Revista']=tituloR
    else :
        content_book['Titulo Revista']=None
    area=a
    if area:
        content_book['Area']=area
    else :
        content_book['Area']=None
    tematica=te
    if tematica:
        content_book['Tematica']=tematica
    else :
        content_book['Tematica']=None
    #titulo del libro
    try:
        titulo=s_item.find('h3').get_text(strip=True)
        content_book['Titulo Articulo']=titulo
    except AttributeError:
        content_book['Titulo Articulo']=None
    try:
        des=s_item.find('div', id='articleAbstract').get_text(strip=True)
        content_book['Resumen']=des.replace("Resumen","")
    except AttributeError:
        content_book['Resumen']=None
    try:
        de=s_item.find('div', class_='panel-boody').get_text(strip=True)
        content_book['Abstract']=de#.replace("Abstract","")
    except AttributeError:
        content_book['Abstract']=None
    try:
        urlA=url
        content_book['Link Articulo']=urlA
    except AttributeError:
        content_book['Link Articulo']=None
        #obtener link PDF
    try:
        ancla_link=s_item.find('div', id='articleFullText').find('a').get('href')
        content_book['Link PDF']=urljoin(url_root,ancla_link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

El list_scraper hace un scrapeo de la cada uno de los artículos, ya que en datos_book se acumuló la información obtenida de lo métodos anteriores.

In [ ]:
list_scraper=list_scraper[0:230]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

### Se obtiene la información requerida de los artículos.

La variable **datos_book** es un listado en el que se recolecto la información requerida por lo cual
se convierte en un **DataFrame**.

In [ ]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

En el caso de esta revista solo de deseaba obtener los artículos, por lo cual
en algunas ocasiones se obtienes cosas diferentes, para esto con la función isin lo que hace es 
eliminar mediante el **Titulo Articulo** los archivos que no se desean.

In [ ]:
df=df_catalogo[~ df_catalogo["Titulo Articulo"].isin(["Presentación","Editorial","Autores","Introducción","Corrigenda","Comentario","Revista completa","Preliminares","Portada e índice","Letter Submission","Carta de presentación de manuscritos","In memoriam","COVER","Code of Ethics","CODIGO DE ÉTICA","EDITORIAL PROCESS","In Memoriam","Complate journal","Preliminary","Preliminaries","Complete Journal","Indización y evaluación de Nova Scientia","Editorial del Décimoprimer Número","Nota Editorial"])]
df

La información solicitada se pasa a un **CSV**.

In [ ]:
df.to_csv('Revista136.csv', index=False)

Final del Código